In [ ]:
# Divide the column species into 'genus' and 'species' and keep only the species name replacing the cells in this column.
df_1['species']=df_1['species'].str.split(' ',expand=True, n=1)[1]
df_1.head(3)

In [ ]:
# Check again null values looking at its percentage.
df_1.stb.missing()